CAMS

- mean, median, std, min, max for NO values by year and year-month

- maps for both results


Yearly

In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from collections import defaultdict

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\CAMS\chem_multlvl\5compile_2003_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store yearly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    ds = xr.open_dataset(file_path)  # Load without specifying chunks
    ds = ds.chunk({'time': 500})  # Rechunk after loading 

    # Extract the year from the 'time' dimension
    year = ds['time'].dt.year.values[0]  # Extract the year from the first time step
    
    # Get the total NO data as a numpy array
    no_data = ds['no'].values  # Numpy array (all time steps)

    # Compute statistics using numpy's nan functions
    mean = np.nanmean(no_data, axis=0)
    median = np.nanmedian(no_data, axis=0)
    std = np.nanstd(no_data, axis=0)
    max_ = np.nanmax(no_data, axis=0)
    min_ = np.nanmin(no_data, axis=0)
    total = np.nansum(no_data, axis=0)
    
    # Combine results into a new dataset
    stats = xr.Dataset({
        'Mean': (['latitude', 'longitude'], mean),
        'Median': (['latitude', 'longitude'], median),
        'Std': (['latitude', 'longitude'], std),
        'Max': (['latitude', 'longitude'], max_),
        'Min': (['latitude', 'longitude'], min_),
        'Total': (['latitude', 'longitude'], total)
    }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

    # Convert to DataFrame and restructure
    stats_df = stats.to_dataframe().reset_index()
    stats_df['Year'] = year  # Add the year directly from time
    stats_df = stats_df.set_index(['Year', 'latitude', 'longitude'])
    
    # Append to list
    df_list.append(stats_df)

# Concatenate all years into a single DataFrame
df_final = pd.concat(df_list)

# Display the DataFrame
print(df_final)


                              Mean    Median         Std          Max  \
Year latitude longitude                                                 
2003 66.0     -12.00      0.533547  0.000000    3.077233    77.458699   
              -11.25      0.432999  0.000000    2.325763    66.558433   
              -10.50      0.400131  0.000000    2.134020    62.236401   
              -9.75       0.395908  0.000000    2.179073    58.867669   
              -9.00       0.374816  0.000000    2.053144    56.153527   
...                            ...       ...         ...          ...   
2024 34.5      33.00      0.773183  0.181296    3.125959    74.875809   
               33.75      0.975634  0.172837    2.424369    55.189026   
               34.50      2.077910  0.170778    7.780618   142.041138   
               35.25      6.868222  0.867099   21.596523   375.764954   
               36.00     71.177551  4.949701  186.170303  1657.417480   

                                  Min          Tot

In [2]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\CAMS\chem_multlvl\yearly_no_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")


Dataset saved to: E:\IPMA\CAMS\chem_multlvl\yearly_no_stats.nc


In [3]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_multlvl\yearly_no_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_multlvl\maps_year\no"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration (µg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Projection (assume data is in PlateCarree, common for lat/lon grids)
data_crs = ccrs.PlateCarree()

# Loop through years
for year in ds['Year'].values:
    fig, ax = plt.subplots(figsize=(8, 6),
                           subplot_kw={'projection': ccrs.PlateCarree()})

    # Select data for current year
    data = ds[var].sel(Year=year)

    # Plot the data
    im = ax.pcolormesh(ds['longitude'], ds['latitude'], data,
                       cmap=cmap, vmin=vmin, vmax=vmax,
                       shading='auto', transform=data_crs)

    # Add features (land, ocean, borders)
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.set_extent([-12, 36, 34, 66], crs=data_crs)  # Adjust to the working area

    # Gridlines (bottom and left only)
    gl = ax.gridlines(draw_labels=True, linewidth=0)
    gl.top_labels = False
    gl.right_labels = False
    gl.bottom_labels = True
    gl.left_labels = True

    # Title and colorbar
    ax.set_title(f'{title} - {year}', fontsize=12)

    # Create a colorbar with more space
    cbar = fig.colorbar(im, ax=ax, orientation='horizontal', label='µg/m³', shrink=0.7, pad=0.1)
    
    # Adjust the colorbar's position
    cbar.ax.yaxis.set_ticks_position('right')  # Put colorbar ticks on the right side

    # Save the plot
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f"mean_concentration_no_{year}.png"), dpi=150)
    plt.close(fig)

print("✅ Mean temperature maps with adjusted colorbar spacing generated!")


✅ Mean temperature maps with adjusted colorbar spacing generated!


Monthly

In [4]:
import xarray as xr
import pandas as pd
import os
import numpy as np

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\CAMS\chem_multlvl\4conversion_2003_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store monthly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    ds = xr.open_dataset(file_path)  # Load without specifying chunks
    ds = ds.chunk({'time': 500})  # Rechunk after loading 
    
    # Extract year and month from the 'time' dimension
    ds.coords['year'] = ds['time'].dt.year
    ds.coords['month'] = ds['time'].dt.month
    
    # Get the total NO data as a numpy array (time, latitude, longitude)
    no_data = ds['no'].values
    
    # Calculate statistics using numpy's nan functions along the 'time' dimension
    mean = np.nanmean(no_data, axis=0)
    median = np.nanmedian(no_data, axis=0)
    std = np.nanstd(no_data, axis=0)
    max_ = np.nanmax(no_data, axis=0)
    min_ = np.nanmin(no_data, axis=0)
    total = np.nansum(no_data, axis=0)
    
    # Create a new xarray Dataset for the statistics
    stats = xr.Dataset({
        'Mean': (['latitude', 'longitude'], mean),
        'Median': (['latitude', 'longitude'], median),
        'Std': (['latitude', 'longitude'], std),
        'Max': (['latitude', 'longitude'], max_),
        'Min': (['latitude', 'longitude'], min_),
        'Total': (['latitude', 'longitude'], total)
    }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})
    
    # Convert to DataFrame and restructure
    stats_df = stats.to_dataframe().reset_index()
    stats_df['Year'] = ds['year'].values[0]  # Add the year directly from time
    stats_df['Month'] = ds['month'].values[0]  # Add the month directly from time
    stats_df = stats_df.set_index(['Year', 'Month', 'latitude', 'longitude'])
    
    # Append to list
    df_list.append(stats_df)

# Concatenate all months into a single DataFrame
df_final = pd.concat(df_list)

# Display the DataFrame
print(df_final)
df_final


                                     Mean     Median         Std         Max  \
Year Month latitude longitude                                                  
2003 1     66.0     -12.00       0.000000   0.000000    0.000000    0.000000   
                    -11.25       0.000000   0.000000    0.000000    0.000000   
                    -10.50       0.000000   0.000000    0.000000    0.000000   
                    -9.75        0.000000   0.000000    0.000000    0.000000   
                    -9.00        0.000000   0.000000    0.000000    0.000000   
...                                   ...        ...         ...         ...   
2024 11    34.5      33.00       1.429546   0.247120    3.659390   25.434452   
                     33.75       0.723807   0.066005    1.148728    9.011481   
                     34.50       1.387258   0.037941    2.966680   22.016918   
                     35.25       4.805223   0.746794    9.414144   68.665276   
                     36.00     123.48697

Mean     Median         Std         Max  \
Year Month latitude longitude                                                  
2003 1     66.0     -12.00       0.000000   0.000000    0.000000    0.000000   
                    -11.25       0.000000   0.000000    0.000000    0.000000   
                    -10.50       0.000000   0.000000    0.000000    0.000000   
                    -9.75        0.000000   0.000000    0.000000    0.000000   
                    -9.00        0.000000   0.000000    0.000000    0.000000   
...                                   ...        ...         ...         ...   
2024 11    34.5      33.00       1.429546   0.247120    3.659390   25.434452   
                     33.75       0.723807   0.066005    1.148728    9.011481   
                     34.50       1.387258   0.037941    2.966680   22.016918   
                     35.25       4.805223   0.746794    9.414144   68.665276   
                     36.00     123.486977  14.177229  194.272293  886.736267   

                                        Min         Total  
Year Month latitude longitude                              
2003 1     66.0     -12.00     0.000000e+00      0.000000  
                    -11.25     0.000000e+00      0.000000  
                    -10.50     0.000000e+00      0.000000  
                    -9.75      0.000000e+00      0.000000  
                    -9.00      0.000000e+00      0.000000  
...                                     ...           ...  
2024 11    34.5      33.00     1.508271e-15    343.091034  
                     33.75     1.493745e-15    173.713715  
                     34.50     1.741935e-15    332.941833  
                     35.25     1.914042e-15   1153.253540  
                     36.00     1.931244e-15  29636.875000  

[735085 rows x 6 columns]

In [5]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\CAMS\chem_multlvl\monthly_no_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")

df_final_xr


Dataset saved to: E:\IPMA\CAMS\chem_multlvl\monthly_no_stats.nc


<xarray.Dataset> Size: 35MB
Dimensions:    (Year: 22, Month: 12, latitude: 43, longitude: 65)
Coordinates:
  * Year       (Year) int64 176B 2003 2004 2005 2006 ... 2021 2022 2023 2024
  * Month      (Month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * latitude   (latitude) float64 344B 34.5 35.25 36.0 36.75 ... 64.5 65.25 66.0
  * longitude  (longitude) float64 520B -12.0 -11.25 -10.5 ... 34.5 35.25 36.0
Data variables:
    Mean       (Year, Month, latitude, longitude) float64 6MB 1.535 ... nan
    Median     (Year, Month, latitude, longitude) float64 6MB 0.0 0.0 ... nan
    Std        (Year, Month, latitude, longitude) float64 6MB 4.946 ... nan
    Max        (Year, Month, latitude, longitude) float64 6MB 42.86 ... nan
    Min        (Year, Month, latitude, longitude) float64 6MB 0.0 0.0 ... nan
    Total      (Year, Month, latitude, longitude) float64 6MB 380.7 ... nan

In [6]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_multlvl\monthly_no_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_multlvl\maps_month\no"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration (µg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Coordinates
lat = ds['latitude']
lon = ds['longitude']

# Loop through all years and months
for year in ds['Year'].values:
    for month in ds['Month'].values:
        data = ds[var].sel(Year=year, Month=month)

        # Skip empty or all-NaN values
        if np.isnan(data).all():
            continue

        # Plotting
        fig = plt.figure(figsize=(7, 6))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                           shading='auto', transform=ccrs.PlateCarree())

        ax.set_title(f'{title} - {year} / {month:02d}', fontsize=12)
        ax.coastlines(resolution='10m', linewidth=0.8)
        ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

        # Gridlines (bottom and left only)
        gl = ax.gridlines(draw_labels=True, linewidth=0)
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = True
        gl.left_labels = True

        # Colorbar
        cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.7, pad=0.1)
        cbar.set_label('µg/m³')
        cbar.ax.tick_params(labelsize=8)

        # Save figure
        plt.tight_layout()
        filename = f"mean_concentration_no_{year}{month:02d}.png"
        plt.savefig(os.path.join(output_folder, filename), dpi=150)
        plt.close(fig)

print("✅ Monthly temperature maps generated for all years and months!")


✅ Monthly temperature maps generated for all years and months!


In [7]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_multlvl\monthly_no_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_multlvl\maps_month\no"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration (µg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Coordinates
lat = ds['latitude']
lon = ds['longitude']

# Loop through all years
for year in ds['Year'].values:
    fig, axs = plt.subplots(3, 4, figsize=(15, 10),
                            subplot_kw={'projection': ccrs.PlateCarree()})
    fig.suptitle(f'{title} - {year}', fontsize=16)

    for i, month in enumerate(ds['Month'].values):
        row, col = divmod(i, 4)
        ax = axs[row, col]

        # Extract data
        data = ds[var].sel(Year=year, Month=month)

        # Skip if no data
        if np.isnan(data).all():
            ax.set_visible(False)
            continue

        ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                           shading='auto', transform=ccrs.PlateCarree())

        ax.set_title(f'Month {month:02d}', fontsize=10)
        ax.coastlines(resolution='10m', linewidth=0.6)
        ax.add_feature(cfeature.BORDERS, linewidth=0.4)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

        # Gridlines (bottom and left only)
        gl = ax.gridlines(draw_labels=True, linewidth=0)
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = row == 2
        gl.left_labels = col == 0

    # Add a single colorbar for all subplots
    cbar_ax = fig.add_axes([0.92, 0.25, 0.015, 0.5])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.set_label('µg/m³')

    # Save figure
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    filename = f"mean_concentration_no_year_{year}.png"
    plt.savefig(os.path.join(output_folder, filename), dpi=150)
    plt.close(fig)

print("✅ Yearly temperature grids generated (3x4 months per year)!")


C:\Users\sofia\AppData\Local\Temp\ipykernel_16728\1761472837.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_16728\1761472837.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_16728\1761472837.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_16728\1761472837.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_16728\1761472837.py:72: UserWarning: This figure includes Ax

✅ Yearly temperature grids generated (3x4 months per year)!
